In [3]:
from scipy.signal import find_peaks
import numpy as np
from LAB_UW_forward_modeling import *
from file_io import *
from signal_processing import *
from synthetic_data import *
import os
import matplotlib.pyplot as plt 
import glob
import pandas as pd

ModuleNotFoundError: No module named 'src'

In [ ]:

def find_mechanical_data(file_path_list, pattern):
    """
    Trova un file specifico all'interno di una lista di percorsi dei file utilizzando un pattern.
    
    Args:
        file_path_list (list): Lista di percorsi dei file in cui cercare il file.
        pattern (str): Pattern per il nome del file da cercare.
    
    Returns:
        str: Percorso completo del file trovato, o None se non viene trovato nessun file corrispondente.
    """
    for file_path in file_path_list:
        if glob.fnmatch.fnmatch(file_path, pattern):
            print("MECHANICAL DATA CHOOSE:", file_path)
            return file_path
    return None  # Nessun file trovato nella lista
    

def find_sync_values(mech_data_path):
    """
    Trova i valori di picco sincronizzazione all'interno di un file di dati meccanici.

    Questa funzione legge un file CSV contenente dati meccanici, estrae la colonna
    relativa alla sincronizzazione e individua i picchi di sincronizzazione in base
    ai parametri specificati.

    Args:
        mech_data_path (str): Percorso del file CSV contenente i dati meccanici.

    Returns:
        numpy.ndarray: Un array NumPy contenente gli indici dei picchi di sincronizzazione
                       trovati nei dati meccanici.
    """
    mech_data = pd.read_csv(mech_data_path, sep=',', skiprows=[1])
    sync_data = mech_data.sync
    
    # Trova i picchi di sincronizzazione nei dati sincronizzazione
    sync_peaks, _ = find_peaks(sync_data, prominence=4.2, height=4)
    print(sync_peaks)
    return mech_data, sync_data, sync_peaks

def plot_sync_peaks(sync_data, sync_peaks, experiment_name):
    """
    Visualizza i picchi di sincronizzazione su un grafico dei dati di sincronizzazione.

    Questa funzione prende i dati di sincronizzazione e gli indici dei picchi di sincronizzazione,
    quindi crea un grafico per visualizzare sia i dati di sincronizzazione che i picchi di sincronizzazione
    evidenziati in rosso.

    Args:
        sync_data (numpy.ndarray): Array NumPy contenente i dati di sincronizzazione.
        sync_peaks (numpy.ndarray): Array NumPy contenente gli indici dei picchi di sincronizzazione.
        experiment_name (str): Nome dell'esperimento o della prova da visualizzare nel titolo del grafico.
    """
    
    plt.figure(figsize=(10, 4))
    plt.title(f'Sync Peaks {experiment_name}')
    

    # Plot dei picchi di sincronizzazione evidenziati in rosso
    plt.scatter(np.arange(0, len(sync_data))[sync_peaks], sync_data[sync_peaks], c='r', s = 10, zorder = 2, alpha = 0.8)
    
    # Plot dei dati di sincronizzazione
    plt.plot(sync_data, zorder = 1, c = 'k', linewidth = 0.8)
    
    plt.ylabel('Arduino voltage [V]', fontsize=12)
    plt.xlabel('records #', fontsize=12)
    
    plt.show()
    



In [ ]:
# Define the color data in Extended Array format
colors = {
    'reseda_green': '#788054',
    'dutch_white': '#E0D6B4',
    'khaki': '#CABB9E',
    'platinum': '#E7E5E2',
    'black_olive': '#322D1E'
}

In [ ]:
# Search Space for global optimization of the velocities
cmin = 600        
cmax = 1800 
c_step = 1
c_gouge_list = np.arange(cmin, cmax,c_step) 


In [ ]:
# Load Waveform data
machine_name = "Brava_2"
experiment_name = "s0108"
data_type_uw = 'data_tsv_files'
data_type_mech = 'mechanical_data'
sync_file_pattern = '*s*_data_rp' #pattern to find specific experiment in mechanical data

#MAKE MECHANICAL PATH LIST
infile_path_list_mech = make_infile_path_list(machine_name, experiment_name, data_type=data_type_mech)
#LOAD MECHANICAL DATA
mech_data_path= find_mechanical_data(infile_path_list_mech, sync_file_pattern)
mech_data, _ , sync_peaks = find_sync_values(mech_data_path)

In [ ]:
dir_path = "/home/michele/Desktop/Dottorato/active_source_implementation/experiments_Brava_2/s0108/data_analysis/global_optimization_velocity/workstation_ones"
font_size = 30

choosen_uw_file = 0
file_name = sorted(os.listdir(dir_path))[choosen_uw_file]
file_path = os.path.join(dir_path,file_name)
l2_norm = np.load(file_path) 
l2_norm = l2_norm[:,:len(c_gouge_list)]

# look for the minimum around cross_corr values
cross_corr_min = 600     # [m/s]  velocity from cross-correlation  
cross_corr_max = 1200     # [m/s]  velocity from cross-correlation  
idx_c_gouge_list_plot =  np.where((c_gouge_list>700) & (c_gouge_list<1000))
c_gouge_list_plot = c_gouge_list[idx_c_gouge_list_plot]
l2_norm_plot = l2_norm[:,idx_c_gouge_list_plot]

right_v = c_gouge_list_plot[np.argmin(l2_norm_plot, axis=2)]
plt.figure()
plt.plot(right_v)   
plt.title(file_name)
plt.ylabel('veocity [m/s]')
plt.xlabel('wave recorded')

number_of_waveforms_wanted = 10
thickness_gouge_1_list = mech_data.rgt_lt_mm[sync_peaks[2*choosen_uw_file]: sync_peaks[2*choosen_uw_file+1]].values/10  #the velocities are in cm/mus, layer thickness in mm. Divided by ten!!!
downsampling = round(len(thickness_gouge_1_list)/number_of_waveforms_wanted)
thickness_gouge_1_list = thickness_gouge_1_list[::downsampling]
right_v = right_v[:len(thickness_gouge_1_list)]
# plt.figure()
# plt.scatter(thickness_gouge_1_list,right_v,s=5)
# plt.title('correlation layer thickness and shear velocity')
# plt.xlabel('layer thickness [cm]')
# plt.ylabel('shear velocity [m/s]')

uw_choosen = 0
fig = plt.figure(figsize=[16, 8])
ax = plt.axes()

# Setting the background color of the plot 
# using set_facecolor() method
ax.set_facecolor('lightsteelblue')
ax.plot(c_gouge_list,l2_norm[uw_choosen,:], color='white', alpha=0.5,linewidth=6.0, label= "UW across iron block")

ax.set_xlim([1000, 1800])
ax.set_xlabel('Shear Wave Velocity [cm/$\mu$s]',fontsize=0.7*font_size)
ax.set_ylabel('List Sqaure of the Residuals',fontsize=0.7*font_size)
plt.title(f' Search of the Minimum for a Recorded Waveform',fontsize=font_size)
plt.xticks(fontsize=0.7*font_size)
plt.yticks(fontsize=0.7*font_size)

# outfile_path_l2norm_EGU = f"/home/michele/Desktop/Conferenza EGU 2024/l2norm_{file_name}.eps"
# plt.savefig(outfile_path_l2norm_EGU)
